In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split

# Setup Paths
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from smartcook.data_gen import LabeledCookingDataset
from smartcook.models import MaskedCookingAutoencoder, CookingPredictor

# --- 1. CONFIGURATION ---
BATCH_SIZE = 32
LEARNING_RATE = 0.01 
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PRETRAINED_PATH = os.path.join('..', 'src', 'smartcook', 'pretrained_encoder.pth')

# --- 2. LOAD PRETRAINED BRAIN ---
print("Loading Pretrained Model...")
base_model = MaskedCookingAutoencoder().to(DEVICE)

# Check if the pretrained file exists
if os.path.exists(PRETRAINED_PATH):
    base_model.load_state_dict(torch.load(PRETRAINED_PATH))
    print("✅ Pretrained weights loaded.")
else:
    print("⚠️ WARNING: Pretrained weights not found. Using random weights (Performance will be worse).")

# Create the Predictor
# This uses the 'base_model.encoder' and freezes it
predictor = CookingPredictor(base_model).to(DEVICE)

# --- 3. PREPARE LABELED DATA ---
# This dataset now returns 3 items: (Input, Stage_Label, Temp_Target)
full_dataset = LabeledCookingDataset(num_samples=1000)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 4. SETUP TRAINING ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(predictor.parameters(), lr=LEARNING_RATE)

# --- 5. FINE-TUNING LOOP ---
print(f"Starting Fine-Tuning on {DEVICE}...")

for epoch in range(EPOCHS):
    predictor.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    # FIX: Unpack 3 values, ignoring the last one (_)
    for inputs, labels, _ in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        
        # The predictor returns (stage_logits, time_pred)
        # We only care about stage_logits for this task
        stage_logits, _ = predictor(inputs)
        
        loss = criterion(stage_logits, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Calculate Accuracy
        _, predicted = torch.max(stage_logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    acc = 100 * correct / total
    
    # Print every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss/len(train_loader):.4f} | Accuracy: {acc:.2f}%")

print("✅ Downstream Task Complete!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading Pretrained Model...
✅ Pretrained weights loaded.
Starting Fine-Tuning on cpu...
Epoch 10/50 | Loss: 0.0028 | Accuracy: 100.00%
Epoch 20/50 | Loss: 0.0010 | Accuracy: 100.00%
Epoch 30/50 | Loss: 0.0006 | Accuracy: 100.00%
Epoch 40/50 | Loss: 0.0003 | Accuracy: 100.00%
Epoch 50/50 | Loss: 0.0002 | Accuracy: 100.00%
✅ Downstream Task Complete!
